In [40]:
# Import libraries.
# Scientific computing.
import pandas as pd
import numpy as np
from scipy import stats

# Visualization tools.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Iteration.
from functools import lru_cache

In [2]:
# Import necessary aggregations for visualizations.
survey_df = pd.read_csv("../Clean_Data_Resources/Survey_Results.csv", dtype={'Course_Code': 'str'})
survey_windows_df = pd.read_csv("../Clean_Data_Resources/Survey_Results_Survey_Windows.csv")
survey_likert_guide_df = pd.read_csv("../Clean_Data_Resources/Survey_Results_Likert_Guide.csv")

/tmp/ipykernel_35505/4016719273.py:2: DtypeWarning: Columns (0: R_Dropin_Tutoring, 1: R_Online_Active_Participation, 2: R_Online_Engaging_Content, 3: R_Weekly_Emails, 4: T_Online_Experience_Suggestions, 5: T_Other_Suggestions) have mixed types. Specify dtype option on import or set low_memory=False.
  survey_df = pd.read_csv("../Clean_Data_Resources/Survey_Results.csv", dtype={'Course_Code': 'str'})


In [3]:
# Initialize a color palette to represent major and minor disciplines offered within Supplemental Instruction.

discipline_colors = {
    'CHEM': '#9D2235',  # Razorback Cardinal Red
    'BIOL': '#2D6A4F',  # Forest Green
    'ACCT': '#1B4965',  # Deep Blue
    'ECON': '#7D4E2D',  # Warm Brown
    # Minor disciplines.
    'NURS': '#4A7BA7',  # Steel Blue
    'WCOB': '#5C8A5E',  # Sage Green
    'EXSC': '#8B5E3C',  # Tan Brown
    'CHEG': '#6B4226',  # Dark Brown
    'MEEG': '#3D7068',  # Teal
    'UNIV': '#7A6B4F',  # Khaki
    'MATH': '#4F6D7A'   # Slate Blue
}

semester_colors = {
    'Fall': '#8B4513',   # Saddle Brown 
    'Spring': '#2D6A4F'  # Forest Green 
}

In [32]:
# Configure Likert themes for viz purposes.

# Agreement scale ordering.
agreement_order = [
    'Strongly disagree',
    'Moderately disagree', 
    'Neither disagree nor agree',
    'Moderately agree',
    'Strongly agree'
]

# Helpfulness scale ordering.
helpfulness_order = [
    'Not at all helpful',
    'Slightly helpful',
    'Moderately helpful',
    'Very helpful',
    'Extremely helpful'
]

# Color schemes for Likert responses.
agreement_colors = {
    'Strongly disagree': '#8B0000',
    'Moderately disagree': '#D4445F',
    'Neither disagree nor agree': '#B8B8B8',
    'Moderately agree': '#7DBF7D',
    'Strongly agree': '#2D6A4F'
}

helpfulness_colors = {
    'Not at all helpful': '#8B0000',
    'Slightly helpful': '#D4445F',
    'Moderately helpful': '#B8B8B8',
    'Very helpful': '#7DBF7D',
    'Extremely helpful': '#2D6A4F'
}

In [4]:
# Count all disciplines,
discipline_counts = survey_df['Discipline'].value_counts().sort_values(ascending=False)

# Split into major and minor disciplines,
major_disciplines = ['CHEM', 'BIOL', 'ACCT', 'ECON']
major_data = discipline_counts[discipline_counts.index.isin(major_disciplines)]
minor_data = discipline_counts[~discipline_counts.index.isin(major_disciplines)]

# Sort both.
major_data = major_data.sort_values(ascending=True)
minor_data = minor_data.sort_values(ascending=True)

In [5]:
# Count disciplines and sort.
discipline_counts = survey_df['Discipline'].value_counts().sort_values(ascending=False)

# Define major disciplines for color coding.
major_disciplines = ['CHEM', 'BIOL', 'ACCT', 'ECON']

# Create colors.
colors = [discipline_colors.get(disc, '#AAAAAA') for disc in discipline_counts.index]

# Create single bar chart.
fig = go.Figure()

fig.add_trace(go.Bar(
    x=discipline_counts.index,
    y=discipline_counts.values,
    marker=dict(
        color=colors,
        line=dict(color='rgba(26, 51, 84, 0.2)', width=0.5),
        opacity=0.9
    ),
    text=discipline_counts.values,
    textposition='outside',
    textfont=dict(family='Noto Sans, sans-serif', size=12, color='#333333'),
    showlegend=False
))

fig.update_layout(
    title=dict(
        text='<b>CHEM, BIOL, ACCT, and ECON Dominate Enrollment</b><br><span style="font-size:16px; color:#666666; font-weight:500">Survey Responses by Discipline from Fall 2019 to Spring 2025</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title='',
        tickfont=dict(family='Noto Sans, sans-serif', size=13, color='#333333'),
        showgrid=False
    ),
    yaxis=dict(
        title=dict(
            text='Number of Responses',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)',
        range=[0, discipline_counts.max() * 1.15]  # Add 15% padding to top
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    margin=dict(l=80, r=80, t=120, b=80)
)

fig.show()

In [6]:
fig3 = go.Figure()

major_disciplines_data = survey_windows_df[survey_windows_df['Discipline'].isin(['CHEM', 'BIOL', 'ACCT', 'ECON'])]

for discipline in ['CHEM', 'BIOL', 'ACCT', 'ECON']:
    data = major_disciplines_data[major_disciplines_data['Discipline'] == discipline]
    fig3.add_trace(go.Box(
        y=data['Collection_Period_Days'],
        name=discipline,
        marker=dict(color=discipline_colors.get(discipline, '#999999')),
        boxmean='sd'
    ))

fig3.update_layout(
    title=dict(
        text='<b>Survey Collection Duration Varies by Discipline</b><br><span style="font-size:16px; color:#666666; font-weight:500">Typical Collection Periods for Major Disciplines</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title='',
        tickfont=dict(family='Noto Sans, sans-serif', size=13, color='#333333'),
        showgrid=False
    ),
    yaxis=dict(
        title=dict(
            text='Collection Period (Days)',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    margin=dict(l=80, r=80, t=120, b=80),
    showlegend=False
)

fig3.show()

In [7]:
semester_year_counts = survey_df.groupby(['Year', 'Semester']).size().reset_index(name='Responses')
semester_year_pivot = semester_year_counts.pivot(index='Year', columns='Semester', values='Responses').fillna(0)

fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x=semester_year_pivot.index,
    y=semester_year_pivot['Fall'],
    name='Fall',
    marker=dict(color=semester_colors['Fall'], opacity=0.9),
    text=semester_year_pivot['Fall'].astype(int),
    textposition='inside',
    textfont=dict(family='Noto Sans, sans-serif', size=11, color='white')
))

fig1.add_trace(go.Bar(
    x=semester_year_pivot.index,
    y=semester_year_pivot['Spring'],
    name='Spring',
    marker=dict(color=semester_colors['Spring'], opacity=0.9),
    text=semester_year_pivot['Spring'].astype(int),
    textposition='inside',
    textfont=dict(family='Noto Sans, sans-serif', size=11, color='white')
))

fig1.update_layout(
    title=dict(
        text='<b>Survey Responses Grew Significantly Post-COVID</b><br><span style="font-size:16px; color:#666666; font-weight:500">Response Volume by Semester, 2019-2025</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title='',
        tickfont=dict(family='Noto Sans, sans-serif', size=13, color='#333333'),
        showgrid=False
    ),
    yaxis=dict(
        title=dict(
            text='Number of Responses',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)'
    ),
    barmode='stack',
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    margin=dict(l=80, r=80, t=120, b=80),
    legend=dict(
        font=dict(family='Noto Sans, sans-serif', size=12),
        orientation='h',
        yanchor='bottom',
        y=-0.15,
        xanchor='center',
        x=0.5
    )
)

fig1.show()

In [8]:
# Duration in minutes, filtered
survey_df['Duration_Minutes'] = survey_df['Duration'] / 60
duration_filtered = survey_df[survey_df['Duration_Minutes'] < 15]['Duration_Minutes']

# Key statistics
median_val = duration_filtered.median()
mean_val = duration_filtered.mean()

fig3 = go.Figure()

# Histogram
fig3.add_trace(go.Histogram(
    x=duration_filtered,
    nbinsx=40,
    name='Response Count',
    marker=dict(
        color='#9D2235',
        line=dict(color='rgba(255,255,255,0.3)', width=0.5),
        opacity=0.8
    )
))


# Median line
fig3.add_vline(
    x=median_val,
    line=dict(color="#505050", width=2, dash='dash')
)

# Mean line
fig3.add_vline(
    x=mean_val,
    line=dict(color="#000000", width=2, dash='dot')
)

fig3.update_layout(
    title=dict(
        text='<b>Most Students Complete Surveys in Under 5 Minutes</b><br><span style="font-size:16px; color:#666666; font-weight:500">Survey Completion Time Distribution</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title=dict(
            text='Duration (Minutes)',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=False,
        range=[0, 15],
        dtick=1 
    ),
    yaxis=dict(
        title=dict(
            text='Number of Responses',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=1000,
    margin=dict(l=100, r=120, t=140, b=100),
    legend=dict(
        font=dict(family='Noto Sans, sans-serif', size=14),
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='right',
        x=0.98,
        bgcolor='rgba(255,255,255,0.8)',
    ),
    barmode='overlay'
)

fig3.show()

In [9]:
# Duration by major discipline
fig3b = go.Figure()

for discipline in major_disciplines:
    disc_data = survey_df[
        (survey_df['Discipline'] == discipline) & 
        (survey_df['Duration_Minutes'] < 15)
    ]['Duration_Minutes']
    
    median_val = disc_data.median()
    mean_val = disc_data.mean()
    
    fig3b.add_trace(go.Histogram(
        x=disc_data,
        nbinsx=40,
        name=discipline,
        marker=dict(
            color=discipline_colors[discipline],
            line=dict(color='rgba(255,255,255,0.3)', width=0.5),
            opacity=0.7
        )
    ))

fig3b.update_layout(
    title=dict(
        text='<b>Completion Times Are Consistent Across Major Disciplines</b><br><span style="font-size:16px; color:#666666; font-weight:500">Survey Completion Time Distribution by Discipline</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title=dict(
            text='Duration (Minutes)',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=False,
        range=[0, 15],
        dtick=1
    ),
    yaxis=dict(
        title=dict(
            text='Number of Responses',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)'
    ),
    barmode='overlay',
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=1000,
    margin=dict(l=100, r=120, t=140, b=100),
    legend=dict(
        font=dict(family='Noto Sans, sans-serif', size=14),
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='right',
        x=0.98,
        bgcolor='rgba(255,255,255,0.8)'
    )
)

fig3b.show()

In [42]:
# Pre-compute rating columns once.
rating_cols = [col for col in survey_df.columns 
               if col.startswith('R_') and not col.endswith('_encoded')]

# Create indexed lookup for faster filtering.
survey_df_indexed = survey_df.set_index(['Discipline', 'Course_Code', 'Semester', 'Year'])


In [98]:
# Create a function to generate a likert chart based on parameters provided.
def create_likert_chart(discipline, course_code, semester=None, year=None, 
                        use_indexed=True, min_responses=5):
    """
    Create a stacked Likert chart for any course/semester/year combination.
    
    Parameters:
    discipline : str
        Discipline code (e.g., 'BIOL', 'CHEM')
    course_code : str
        Course code (e.g., '10103')
    semester : str, optional
        'Fall', 'Spring', or None for all semesters
    year : int, optional
        Year (e.g., 2025) or None for all years
    use_indexed : bool
        Use pre-indexed dataframe for faster lookup (default True)
    min_responses : int
        Minimum responses required to generate chart (default 5)
    
    Returns:
    plotly.graph_objects.Figure or None
    """
    
    # Fast filtering using index.
    if use_indexed:
        try:
            if semester and year:
                course_data = survey_df_indexed.loc[(discipline, course_code, semester, year), :].copy()
                if isinstance(course_data, pd.Series):
                    course_data = course_data.to_frame().T
            elif semester:
                course_data = survey_df_indexed.loc[(discipline, course_code, semester, slice(None)), :].copy()
            elif year:
                course_data = survey_df_indexed.loc[(discipline, course_code, slice(None), year), :].copy()
            else:
                course_data = survey_df_indexed.loc[(discipline, course_code, slice(None), slice(None)), :].copy()
            course_data = course_data.reset_index()
        except KeyError:
            return None
    else:
        # Standard filtering.
        mask = (survey_df['Discipline'] == discipline) & (survey_df['Course_Code'] == course_code)
        if semester:
            mask &= (survey_df['Semester'] == semester)
        if year:
            mask &= (survey_df['Year'] == year)
        course_data = survey_df[mask].copy()
    
    n_responses = len(course_data)
    if n_responses < min_responses:
        return None
    
    # Get rating columns with data.
    rating_cols_with_data = [col for col in rating_cols
                             if course_data[col].notna().sum() > 0]
    
    if len(rating_cols_with_data) == 0:
        return None
    
    # Determine scale type.
    first_col = rating_cols_with_data[0]
    first_vals = course_data[first_col].dropna()
    if len(first_vals) == 0:
        return None
    
    scale_type = 'helpfulness' if 'helpful' in str(first_vals.iloc[0]).lower() else 'agreement'
    
    # Select scale configuration.
    scale_order = helpfulness_order if scale_type == 'helpfulness' else agreement_order
    scale_colors = helpfulness_colors if scale_type == 'helpfulness' else agreement_colors
    
    # Pre-allocate data structure.
    data_by_category = {cat: [] for cat in scale_order}
    y_labels = []
    
    # Vectorized processing.
    for col in rating_cols_with_data:
        clean_label = col.replace('R_', '').replace('_', ' ')
        y_labels.append(clean_label)
        
        # Fast value counts.
        counts = course_data[col].value_counts()
        total = counts.sum()
        
        if total > 0:
            # Compute percentages for all categories at once.
            pct_dict = ((counts / total * 100).round(1)).to_dict()
            for cat in scale_order:
                data_by_category[cat].append(pct_dict.get(cat, 0))
        else:
            for cat in scale_order:
                data_by_category[cat].append(0)
    
    # Build title.
    title_parts = [f'<b>{discipline} {course_code} Likert Assessments for']
    if semester and year:
        title_parts.append(f' {semester} {year}')
    elif semester:
        title_parts.append(f' {semester} (All Years)')
    elif year:
        title_parts.append(f' {year} (Both Semesters)')
    else:
        title_parts.append(' All Semesters & Years')
    
    title_parts.append(f'</b><br><span style="font-size:18px; color:#666666; font-weight:500">Survey Responses (n={n_responses})</span>')
    title_text = ''.join(title_parts)
    
    # Create figure
    fig = go.Figure()
    
    for category in scale_order:
        fig.add_trace(go.Bar(
            y=y_labels,
            x=data_by_category[category],
            name=category,
            orientation='h',
            marker=dict(color=scale_colors[category]),
            text=[f"{val:.1f}%" if val > 3 else '' for val in data_by_category[category]],
            textposition='inside',
            textfont=dict(color='white', size=12),
            hovertemplate='%{y}<br>' + category + ': %{x:.1f}%<extra></extra>'
        ))
    
    fig.update_layout(
        title=dict(
            text=title_text,
            font=dict(family='Noto Sans, sans-serif', size=30, color='#1a1a1a'),
            x=0,
            xanchor='left',
            pad=dict(l=20)
        ),
        xaxis=dict(
            title='Percentage',
            tickfont=dict(family='Noto Sans, sans-serif', size=14, color='#666666'),
            showgrid=True,
            gridcolor='rgba(224, 224, 224, 0.3)',
            range=[0, 100]
        ),
        yaxis=dict(
            title='',
            tickfont=dict(family='Noto Sans, sans-serif', size=14, color='#333333'),
            showgrid=False,
            ticksuffix='    '  
        ),
        barmode='stack',
        plot_bgcolor='white',
        paper_bgcolor='white',
        height=max(800, len(y_labels) * 50), 
        width=1200,  # Wider
        margin=dict(l=300, r=100, t=120, b=140),  # More bottom margin for legend
        legend=dict(
            font=dict(family='Noto Sans, sans-serif', size=11),
            orientation='h',
            yanchor='top',
            y=-0.10,  # Position below chart
            xanchor='center',
            x=0.5
        )
    )
    
    return fig

In [100]:
survey_df.head(10)

,Start_Date,End_Date,Duration,Recorded_Date,Response_ID,Source_File,Discipline,Course_Code,Is_Online,Semester,...,R_Leader_Knows_Class_Content_encoded,R_Leader_Subject_Competence_encoded,R_Leader_Willing_To_Help_encoded,R_Online_Active_Participation_encoded,R_Online_Engaging_Content_encoded,R_Opportunity_Ask_Questions_encoded,R_Overall_Session_Helpfulness_encoded,R_Practice_Problems_encoded,R_Weekly_Emails_encoded,Duration_Minutes
0,2019-11-06 09:03:00,2019-11-06 09:03:00,29,2019-11-06 09:03:00,R_2QlC06WEVwEXtjQ,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,NaN,0.483333
1,2019-11-06 10:06:00,2019-11-06 10:08:00,88,2019-11-06 10:08:00,R_VPgtpoWlVBH0E81,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,4.0,3.0,5.0,5.0,1.0,4.0,4.0,NaN,1.466667
2,2019-11-06 12:18:00,2019-11-06 12:18:00,35,2019-11-06 12:18:00,R_sHdb8bWznRjZ6yB,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,5.0,5.0,NaN,4.0,5.0,NaN,0.583333
3,2019-11-06 16:51:00,2019-11-06 16:52:00,47,2019-11-06 16:52:00,R_2e9h2yDUhHWatJZ,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,0.783333
4,2019-11-06 18:38:00,2019-11-06 19:09:00,1862,2019-11-06 19:09:00,R_OrhB1M1BP1Aodr3,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,31.033333
5,2019-11-10 20:22:00,2019-11-10 20:28:00,313,2019-11-10 20:28:00,R_26msq7tdw9SPD57,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,NaN,5.216667
6,2019-11-06 10:51:00,2019-11-10 21:05:00,382442,2019-11-10 21:05:00,R_vpCTT9utueubbuF,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,NaN,6374.033333
7,2019-11-10 22:51:00,2019-11-10 22:52:00,84,2019-11-10 22:52:00,R_DwxKI8TZUcSwnUl,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,3.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,NaN,1.400000
8,2019-11-11 09:08:00,2019-11-11 09:09:00,71,2019-11-11 09:09:00,R_1mVkBt7PM4CbqC2,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,3.0,3.0,4.0,3.0,2.0,3.0,NaN,1.183333
9,2019-11-11 09:16:00,2019-11-11 09:21:00,278,2019-11-11 09:21:00,R_31sYicUWhVfK396,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,4.633333


In [108]:
fig1 = create_likert_chart('EXSC', '33503', 'Spring', 2019)
if fig1:
    fig1.show()

/tmp/ipykernel_35505/2165954668.py:29: PerformanceWarning:

indexing past lexsort depth may impact performance.

